In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy.interpolate import interp1d

# --- Provided Functions ---

def yang_zhang(price_data, window=30, trading_periods=252):
    """
    Calculate realized volatility using the Yang–Zhang method.
    price_data must contain the columns: 'Open', 'High', 'Low', 'Close'
    """
    log_ho = np.log(price_data['High'] / price_data['Open'])
    log_lo = np.log(price_data['Low'] / price_data['Open'])
    log_co = np.log(price_data['Close'] / price_data['Open'])
    
    log_oc = np.log(price_data['Open'] / price_data['Close'].shift(1))
    log_oc_sq = log_oc ** 2
    
    log_cc = np.log(price_data['Close'] / price_data['Close'].shift(1))
    log_cc_sq = log_cc ** 2
    
    rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)
    
    close_vol = log_cc_sq.rolling(window=window).sum() / (window - 1.0)
    open_vol  = log_oc_sq.rolling(window=window).sum() / (window - 1.0)
    window_rs = rs.rolling(window=window).sum() / (window - 1.0)
    
    k = 0.34 / (1.34 + ((window + 1) / (window - 1)))
    result = (open_vol + k * close_vol + (1 - k) * window_rs).apply(np.sqrt) * np.sqrt(trading_periods)
    return result.iloc[-1]

def build_term_structure(days, ivs):
    """
    Build a linear spline term structure from days-to-expiry and corresponding ATM IV values.
    Returns a function that interpolates IV for a given day.
    """
    days = np.array(days)
    ivs = np.array(ivs)
    sort_idx = days.argsort()
    days = days[sort_idx]
    ivs = ivs[sort_idx]
    spline = interp1d(days, ivs, kind='linear', fill_value="extrapolate")
    
    def term_spline(dte):
        if dte < days[0]:
            return ivs[0]
        elif dte > days[-1]:
            return ivs[-1]
        else:
            return float(spline(dte))
    return term_spline

# --- Helper Functions ---

def filter_dates(dates):
    """
    Given a list of expiration dates as strings ('YYYY-MM-DD'),
    return only those dates that are at least 45 days in the future.
    """
    today = datetime.today().date()
    cutoff_date = today + timedelta(days=45)
    sorted_dates = sorted(datetime.strptime(date, "%Y-%m-%d").date() for date in dates)
    return [d.strftime("%Y-%m-%d") for d in sorted_dates if d >= cutoff_date]

def get_sp500_tickers():
    """
    Scrape S&P 500 tickers from Wikipedia.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500_table = tables[0]
    tickers = sp500_table['Symbol'].tolist()
    # Replace periods with dashes if necessary (e.g. BRK.B -> BRK-B)
    tickers = [ticker.replace('.', '-') for ticker in tickers]
    return tickers

def screen_iv30_rv30(ticker):
    """
    For a given ticker, compute iv30_rv30 as:
         iv30_rv30 = term_spline(30) / yang_zhang(price_history)
    Returns the ratio, or None if data is unavailable.
    """
    try:
        stock = yf.Ticker(ticker)
        # If no options data exists, skip
        if len(stock.options) == 0:
            return None
        # Filter expiration dates: we only need those with a minimum of 45 days to expiry.
        exp_dates = filter_dates(stock.options)
        if not exp_dates:
            return None
        
        atm_iv = {}
        # Get underlying current price
        underlying_price = stock.history(period='1d')['Close'].iloc[-1]
        
        # Loop through expiration dates and build ATM IV dictionary
        for exp_date in exp_dates:
            try:
                chain = stock.option_chain(exp_date)
            except Exception:
                continue
            calls = chain.calls
            puts = chain.puts
            if calls.empty or puts.empty:
                continue
            # Find the strike closest to the underlying price for both calls and puts
            calls['abs_diff'] = (calls['strike'] - underlying_price).abs()
            puts['abs_diff'] = (puts['strike'] - underlying_price).abs()
            call_idx = calls['abs_diff'].idxmin()
            put_idx = puts['abs_diff'].idxmin()
            call_iv = calls.loc[call_idx, 'impliedVolatility']
            put_iv = puts.loc[put_idx, 'impliedVolatility']
            atm_iv_value = (call_iv + put_iv) / 2.0
            atm_iv[exp_date] = atm_iv_value
        
        if not atm_iv:
            return None
        
        # Build lists of days to expiration and corresponding ATM IV values.
        today = datetime.today().date()
        dtes = []
        ivs = []
        for exp_date, iv in atm_iv.items():
            exp_date_obj = datetime.strptime(exp_date, "%Y-%m-%d").date()
            days_to_expiry = (exp_date_obj - today).days
            dtes.append(days_to_expiry)
            ivs.append(iv)
        
        # Build the term structure interpolation function
        term_spline = build_term_structure(dtes, ivs)
        iv30 = term_spline(30)
        
        # Get historical price data and compute realized volatility using Yang-Zhang method.
        price_history = stock.history(period='3mo')[['Open','High','Low','Close']].dropna()
        if price_history.empty:
            return None
        rv30 = yang_zhang(price_history)
        if rv30 == 0:
            return None
        iv30_rv30 = iv30 / rv30
        return iv30_rv30
    except Exception:
        return None

# --- Main Screening Script ---

def main():
    sp500_tickers = get_sp500_tickers()
    passed_tickers = {}
    
    print("Screening S&P 500 companies for iv30_rv30 > 1.25...\n")
    
    for ticker in sp500_tickers:
        print(f"Processing {ticker}...")
        ratio = screen_iv30_rv30(ticker)
        if ratio is not None and ratio > 1.25:
            passed_tickers[ticker] = ratio
    
    print("\nTickers with iv30_rv30 > 1.25:")
    for ticker, ratio in passed_tickers.items():
        print(f"{ticker}: {ratio:.2f}")

if __name__ == "__main__":
    main()


Screening S&P 500 companies for iv30_rv30 > 1.25...

Processing MMM...
Processing AOS...
Processing ABT...
Processing ABBV...
Processing ACN...
Processing ADBE...
Processing AMD...
Processing AES...
Processing AFL...
Processing A...
Processing APD...
Processing ABNB...
Processing AKAM...
Processing ALB...
Processing ARE...
Processing ALGN...
Processing ALLE...
Processing LNT...
Processing ALL...
Processing GOOGL...
Processing GOOG...
Processing MO...
Processing AMZN...
Processing AMCR...
Processing AEE...
Processing AEP...
Processing AXP...
Processing AIG...
Processing AMT...
Processing AWK...
Processing AMP...
Processing AME...
Processing AMGN...
Processing APH...
Processing ADI...
Processing ANSS...
Processing AON...
Processing APA...
Processing APO...
Processing AAPL...
Processing AMAT...
Processing APTV...
Processing ACGL...
Processing ADM...
Processing ANET...
Processing AJG...
Processing AIZ...
Processing T...
Processing ATO...
Processing ADSK...
Processing ADP...
Processing AZO.